# Indexing 

In [1]:
import nlp
eli5 = nlp.load_dataset('eli5')

In [2]:
from lfqa_utils import *
from eli5_utils import make_es_index_snippets, query_es_index

In [3]:
wiki40b_snippets = nlp.load_dataset('wiki_snippets', name='wiki40b_en_100_0')['train']

In [5]:
eli5['train_eli5'][1111]

{'q_id': '8ywoey',
 'title': 'Why are mice associated with cheese?',
 'selftext': '[removed]',
 'document': '',
 'subreddit': 'explainlikeimfive',
 'answers': {'a_id': ['e2efn2o', 'e2ecdgh', 'e2eqfo5'],
  'text': ["Most mice prefer to feed on grains and nuts rather than cheese, but when they're hungry, they'll eat whatever they can find that is edible. Cheese tends to smell very strongly, especially when not refrigerated, and that makes it easier to find by mice, so when they're hungry, they'll go for whatever's easiest to find, i.e. cheese. Over time, this created the (incorrect) assumption that mice like cheese, and this in turn was used in cartoons and other media so often that some people assume that mice are inordinately fond of cheese.",
   'Cheese is often storded for long periods of time which increase the chance of mice finding it. Cheese has been made a long time so it has has time to fester as a symbol',
   "Once upon a time, we didn't have refrigerators. We had bread boxes,

In [6]:
wiki40b_snippets.shape

(17553713, 10)

In [7]:
wiki40b_snippets[1111]

{'_id': '{"nlp_id": 258, "wiki_id": "Q16746309", "sp": 6, "sc": 1102, "ep": 6, "ec": 1682}',
 'article_title': 'A Fair to Remember (Modern Family)',
 'end_character': 1682,
 'end_paragraph': 6,
 'nlp_id': 258,
 'passage_text': "sings amazingly. Phil feels frustrated to learn that Claire has loved Harold's singing and does not want to sing anymore, so he goes back home with Claire who surprises him with Chinese acrobats. Phil loves Claire's gift, and Claire manages for the first time to give him a gift which is actually good.\nManny participates in the cake-baking competition, something that Jay does not like since the only participants are old ladies and he believes that Manny does not fit there. Gloria, to get rid of Jay, signs him up to be a fair cop. Gloria, realizing later that Jay was",
 'section_title': 'Plot',
 'start_character': 1102,
 'start_paragraph': 6,
 'wiki_id': 'Q16746309'}

In [8]:
es_client = Elasticsearch([{'host': 'localhost', 'port': '9200'}])

In [9]:
if not es_client.indices.exists('wiki40b_snippets_100w'):
    make_es_index_snippets(es_client, wiki40b_snippets, index_name='wiki40b_snippets_100w')

100%|██████████| 17553713/17553713 [2:49:25<00:00, 1726.75docs/s]  

Indexed 17553713 documents


In [12]:
question = eli5['test_eli5'][1111]

In [13]:
question

{'q_id': '4fpi79',
 'title': 'what makes a professional a professional.',
 'selftext': "Licensed professions aside (physician, EMT, contractor, plumber, etc) what makes a professional a professional?  \n\nSpecifically, what qualifications define a professional photographer?  If someone pays you to take portraits, have you crossed the threshold into professionalism?  If it's your sole source of income (even if it drives you into ), does *that* make you a professional?",
 'document': '',
 'subreddit': 'explainlikeimfive',
 'answers': {'a_id': ['d2auorm', 'd2auwkn', 'd2az1xy', 'd2aurrm'],
  'text': ['The truth is that there\'s no universal standard. For some people it means "I make my living doing this." For others it means "I am highly skilled at this." Unless there is a licensing agency, a guild or union, or a certification test, it could mean a lot of different things.',
   'I would say when you have been paid for something then technically you are a professional at said thing.',
   "T

In [14]:
doc, res_list = query_es_index(question['title'], es_client, index_name='wiki40b_snippets_100w')

In [16]:
res_list

[{'article_title': 'Professional Technologist',
  'end_character': 712,
  'end_paragraph': 4,
  'nlp_id': 61439,
  'section_title': '',
  'start_character': 0,
  'start_paragraph': 2,
  'wiki_id': 'Q7248040',
  'passage_id': '{"nlp_id": 61439, "wiki_id": "Q7248040", "sp": 2, "sc": 0, "ep": 4, "ec": 712}',
  'score': 40.70621,
  'passage_text': "Professional Technologist A Professional Technologist (P. Tech) is a class of certification of Engineering Technologist in Canada.\nThe title of P. Tech is protected across Canada by copyright law, and specifically in certain jurisdictions by provincial law. The P. Tech designation came into place to align Technologists with P. Eng's (Professional Engineers).  Technologists can perform the functions of a Technician, Engineer or Scientist.\nThe title is not used consistently across Canada. In some cases, the P. Tech designation is equivalent to a Certified Engineering Technologist designation. However; in other cases, provincial legislation makes

In [18]:
df = pd.DataFrame({
    'Article': ['---'] + [res['article_title'] for res in res_list],
    'Sections': ['---'] + [res['section_title'] if res['section_title'].strip() != '' else res['article_title']
                 for res in res_list],
    'Text': ['--- ' + question['title']] + [res['passage_text'] for res in res_list],
})

In [19]:
df

,Article,Sections,Text
0,---,---,--- what makes a professional a professional.
1,Professional Technologist,Professional Technologist,"Professional Technologist A Professional Technologist (P. Tech) is a class of certification of Engineering Technologist in Canada.\nThe title of P. Tech is protected across Canada by copyright law, and specifically in certain jurisdictions by provincial law. The P. Tech designation came into place to align Technologists with P. Eng's (Professional Engineers). Technologists can perform the functions of a Technician, Engineer or Scientist.\nThe title is not used consistently across Canada. In some cases, the P. Tech designation is equivalent to a Certified Engineering Technologist designation. However; in other cases, provincial legislation makes Professional Technologists into licensees under the local professional"
2,Galliyan,Critical reception,"stated ""Her voice sounds as polished as that of a professional singer and she makes sure to add an edge to Teri Galliyan""."
3,Happy slapping,France,"violent crimes, including severe beatings and rape, with being an accomplice of such crimes. The law makes it illegal to broadcast the images of such crimes, punishable by up to 5 years in prison and/or a €75,000 fine.\nThe law does not apply to those who took the above actions in order to obtain evidence in court, or as professional journalism. Professional journalism is delimited in France by the ""press card"", which is awarded by a commission representing journalist unions and press organizations. As defined by law, a professional journalist is one whose main activity is professional paid journalism.\nThe bill was"
4,Peter Bondra,National Hockey League,"announced his retirement from professional hockey at the age of 39. Since retirement, Bondra has represented Colosseo USA, a Slovak company that makes custom video scoreboards."
5,Amateur sports,Open union,"swiftly grown to embrace the professional game with many league players joining union to take a slice of the larger amounts of money available in the sport.\nNowadays, while rugby union no longer makes the professional-amateur distinction, the professional-amateur split still exists within rugby league with the British Amateur Rugby League Association (BARLA) strictly amateur, though it allows some ex-professionals to play provided they are no longer under contract. The most recent club to get a ban for fielding a contracted professional was Brighouse Rangers who were expelled from the National Conference League during 2007-2008 season, and the player handed a"
6,Anne X. Alpern,Later career and legacy,"Anne X. Alpern Award, which is awarded annually to a female jurist who demonstrates excellence in the legal profession and makes a significant professional impact on women in the law."
7,SC Veendam,History,"SC Veendam History The club was founded on 4 September 1894, which makes it one of the oldest professional Dutch football clubs. The club's first name was Look-Out, which was changed to Prinses Juliana Veendam (Princess Juliana Veendam) in 1909, which soon became simply Veendam. Veendam became professional in 1954, when the Dutch professional league was introduced. The professional branch of the club left the club in 1974, to form Sportclub Veendam. The amateur branch became Veendam 1894. Sportclub Veendam would later become Betaalde Voetbal Veendam, more commonly known as BV Veendam.\nThe club has spent the largest part of its"
8,Litblog,Examples & Professional,"same principle: to criticize a prize, a Shadow Literary Prize Jury makes its choice before the Real Jury makes its announcement. Professional Professional critics have litblogs at House of Mirth, About Last Night, Inside Higher Ed, The Dizzies, and more. In addition, the litbloggers from The Elegant Variation, Laila Lalami, The Old Hag, Maud Newton, Return of the Reluctant, Bookslut, Ready Steady Book, The Reading Experience, and Sarah Weinman are a few of the bloggers writing book reviews 

# Evaluating Retrieval 

In [22]:
eli5['test_eli5']['answers'][0]

{'a_id': ['dfaqxlx'],
 'text': ["I think it's because, at that moment, it's basically a large number of people all of the same mind and thinking and saying the same thing, which is an astonishing thing if you think about it. \n\nAs a performing musician myself, the biggest rush I get on stage is when people start singing along to the song and, after the initial mismatch in their tempo and volume, they all synchronize and the whole place resonates. That feeling of collective output is an amazing feeling and it really gives you a strong sense of fraternity and belonging, knowing that all the people around you are passionate about the same thing. \n\nIf that's not enough to give a person goosebumps, I'm not sure what is."],
 'score': [2]}

In [25]:
eli5['test_eli5']['title']

['Why do you get chills/goosebumps from hearing large crowds sing along to songs?',
 'How did studded leather and heavy eye makeup come to be the Hollywood dress code for dystopian, post-apocalyptic societies?',
 "What's the difference between a bush, a shrub, and a tree?",
 'Why is it hard to breathe with a strong air gust blowing straight at your face?',
 'how having hereditary cancer genes doesn’t necessarily mean you’ll get cancer? How can you not get cancer if you have these genes?',
 'Why is the Super Bowl played on Sunday night rather than Saturday night?',
 'Why do we use a different letter "a" when typing as opposed to when writing?',
 "How is it possible for wives to 'take' things in a divorce; don't both parties have to agree on how things are split?",
 'How do we know that the colors we perceive are what other people see too?',
 'What is the differences between airplanes and jets',
 'How internet speeds work, megabits vs megabytes, download vs. upload etc.',
 "How to explai

In [26]:
# We first select high-scoring answers (answers beyond the first must have a score of at least 3)
test_qa_list = [(exple['title'],
                ' '.join([a 
                          for i, (a, sc) in enumerate(zip(exple['answers']['text'], exple['answers']['score'])) \
                          if i == 0 or sc >= 3
                         ]))
                for exple in eli5['test_eli5']]

In [27]:
test_qa_list

[('Why do you get chills/goosebumps from hearing large crowds sing along to songs?',
  "I think it's because, at that moment, it's basically a large number of people all of the same mind and thinking and saying the same thing, which is an astonishing thing if you think about it. \n\nAs a performing musician myself, the biggest rush I get on stage is when people start singing along to the song and, after the initial mismatch in their tempo and volume, they all synchronize and the whole place resonates. That feeling of collective output is an amazing feeling and it really gives you a strong sense of fraternity and belonging, knowing that all the people around you are passionate about the same thing. \n\nIf that's not enough to give a person goosebumps, I'm not sure what is."),
 ('How did studded leather and heavy eye makeup come to be the Hollywood dress code for dystopian, post-apocalyptic societies?',
  'I like to think that leather clothing is rather more durable and easy to fix than 

In [28]:
# We then compute word frequencies in answer text
answer_doc_freq = {}
for q, a in test_qa_list:
    for w in a.lower().split():
        answer_doc_freq[w] = answer_doc_freq.get(w, 0) + 1

In [29]:
answer_doc_freq

{'i': 31771,
 'think': 6571,
 "it's": 22221,
 'because,': 141,
 'at': 22063,
 'that': 82456,
 'moment,': 78,
 'basically': 2897,
 'a': 156007,
 'large': 2514,
 'number': 2980,
 'of': 137417,
 'people': 18852,
 'all': 17736,
 'the': 304639,
 'same': 8846,
 'mind': 895,
 'and': 136173,
 'thinking': 762,
 'saying': 1102,
 'thing,': 425,
 'which': 15509,
 'is': 100610,
 'an': 20749,
 'astonishing': 7,
 'thing': 3424,
 'if': 30331,
 'you': 75788,
 'about': 12350,
 'it.': 6582,
 'as': 37520,
 'performing': 125,
 'musician': 40,
 'myself,': 60,
 'biggest': 524,
 'rush': 144,
 'get': 15175,
 'on': 36100,
 'stage': 338,
 'when': 19998,
 'start': 2745,
 'singing': 84,
 'along': 1132,
 'to': 171774,
 'song': 344,
 'and,': 419,
 'after': 4778,
 'initial': 281,
 'mismatch': 7,
 'in': 87777,
 'their': 18679,
 'tempo': 26,
 'volume,': 50,
 'they': 44181,
 'synchronize': 6,
 'whole': 1981,
 'place': 1678,
 'resonates.': 3,
 'feeling': 657,
 'collective': 61,
 'output': 287,
 'amazing': 214,
 'it': 690

In [30]:
# The IDF-recall function is then:
def da_idf_recall(doc, answer):
    d_words = dict([(w, True) for w in doc.lower().split()])
    a_words = answer.lower().split()   
    recall = sum([1. / math.log(1 + answer_doc_freq.get(w, 1)) for w in a_words if w in d_words]) / \
                sum([1. / math.log(1 + answer_doc_freq.get(w, 1)) for w in a_words])
    return recall

In [31]:
def sparse_ret_for_eval(question, n_ret):
    _, sparse_res_list = query_es_index(
        question, es_client, index_name='wiki40b_snippets_100w', n_results=n_ret
    )
    sparse_doc = ' '.join([res['passage_text'] for res in sparse_res_list])
    return sparse_doc

In [32]:
sparse_score = evaluate_retriever(test_qa_list, sparse_ret_for_eval, da_idf_recall)

In [33]:
sparse_score['idf_recall']

0.32116140786999253

In [34]:
sparse_score['retrieval_time']

0.38145983080056256

# Answer Generation 

In [42]:
qa_s2s_tokenizer = AutoTokenizer.from_pretrained('yjernite/bart_eli5')
qa_s2s_model = AutoModelForSeq2SeqLM.from_pretrained('yjernite/bart_eli5').to('cpu')
_ = qa_s2s_model.eval()

In [43]:
qa_s2s_model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
   

In [45]:
questions = []
answers = []

for i in [1111] + [j for j in range(10)]:
    # create support document with the sparse index
    question = eli5['test_eli5'][i]['title']
    doc, res_list = query_es_index(
        question, es_client, index_name='wiki40b_snippets_100w'
    )
    # concatenate question and support document into BART input
    question_doc = "question: {} context: {}".format(question, doc)
    # generate an answer with beam search
    answer = qa_s2s_generate(
            question_doc, qa_s2s_model, qa_s2s_tokenizer,
            num_answers=1,
            num_beams=8,
            min_len=64,
            max_len=256,
            max_input_length=1024,
            device="cpu"
    )[0]
    questions += [question]
    answers += [answer]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [46]:
df = pd.DataFrame({
    'Question': questions,
    'Answer': answers,
})
df.style.set_properties(**{'text-align': 'left'})

,Question,Answer
0,what makes a professional a professional.,"A professional is someone who is paid well for their work. It's not a job, it's a skill. If you're good at it, you're a professional, if you're bad at it you're not a professional. Professional athletes are the best at what they do, and they're paid well to do it."
1,Why do you get chills/goosebumps from hearing large crowds sing along to songs?,"I don't get chills from large crowds singing along to songs, but I do get goosebumps when I see a large group of people sing along to a song. I think it has something to do with the fact that our brains are wired to react to other people singing along with us. It's the same reason why when you see a bunch of people dancing, it makes you happy."
2,"How did studded leather and heavy eye makeup come to be the Hollywood dress code for dystopian, post-apocalyptic societies?","Studded leather and heavy eye makeup have been around for a long time. It's not a new thing, it's just that it's not as common as it used to be. In the 80s and 90s, it was common for actors to wear a lot of makeup, but it wasn't until the 00s and 00s that people started wearing a lot more makeup."
3,"What's the difference between a bush, a shrub, and a tree?","A tree is a living thing. A shrub is just a living part of the soil. A bush is a type of shrub. A flower is a part of a flowering plant. The difference between a shrub and a tree is that shrubs tend to be smaller and have a shorter growing season, while trees tend to have a longer growing season."
4,Why is it hard to breathe with a strong air gust blowing straight at your face?,"It's not hard to breathe with a strong air gust blowing straight at your face. It's hard to breath when the wind is blowing in the opposite direction. The air in your lungs is trying to go in the other direction, so it's hard for your lungs to get the air they're trying to get out."
5,how having hereditary cancer genes doesn’t necessarily mean you’ll get cancer? How can you not get cancer if you have these genes?,"Having hereditary cancer genes doesn’t necessarily mean you’ll get cancer, it just means you have a higher chance of getting cancer. It’s the same as having a recessive gene that causes you to be less likely to have children. If you have recessive genes that cause you to have less children, then you are more likely to get cancer."
6,Why is the Super Bowl played on Sunday night rather than Saturday night?,"The Super Bowl is played on a Sunday night because it is the last game of the regular season. The NFL is a business, and they want to make as much money as possible, so they play the Super Bowl on Sunday night. If they played it on Saturday night, they wouldn't be able to sell as many tickets as they do."
7,"Why do we use a different letter ""a"" when typing as opposed to when writing?","When you write, you're using your whole hand. When you type, you only use a small portion of your hand. It's easier for your brain to distinguish the difference between typing and writing if you use the same hand to do both at the same time, so it makes sense to do it that way."
8,How is it possible for wives to 'take' things in a divorce; don't both parties have to agree on how things are split?,"It is possible for a husband to take things in a divorce. It is also possible for the wife to ""take"" things in divorce. There is no legal requirement that they have to agree on how things are split. If they do not, then the divorce is not legally binding, and they are free to take whatever they want."
9,How do we know that the colors we perceive are what other people see too?,"We don't. There is no way to prove or disprove that the colors we see are the same as the colors other people see. The only way to know for sure is to look at other people's eyes and see if they see the same thing you do. If they do, then you can be pretty confident that your vision is correct."


# ROUGE Scores 

In [48]:
import tqdm

In [49]:
predicted = []
reference = []

# Generate answers for the full test set
for i in tqdm.notebook.tqdm(range(eli5['test_eli5'].num_rows)):
    # create support document with the dense index
    question = eli5['test_eli5'][i]['title']
    doc, res_list = query_es_index(
        question, es_client, index_name='wiki40b_snippets_100w'
    )
    # concatenate question and support document into BART input
    question_doc = "question: {} context: {}".format(question, doc)
    # generate an answer with beam search
    answer = qa_s2s_generate(
            question_doc, qa_s2s_model, qa_s2s_tokenizer,
            num_answers=1,
            num_beams=8,
            min_len=96,
            max_len=256,
            max_input_length=1024,
            device="cpu"
    )[0]
    predicted += [answer]
    reference += [eli5['test_eli5'][i]['answers']['text'][0]]

KeyboardInterrupt: 

In [ ]:
nlp_rouge = nlp.load_metric('rouge')

scores = nlp_rouge.compute(
    predicted, reference,
    rouge_types=['rouge1', 'rouge2', 'rougeL', 'rougeLsum'],
    use_agregator=True, use_stemmer=False
)
df = pd.DataFrame({
    'rouge1': [scores['rouge1'].mid.precision, scores['rouge1'].mid.recall, scores['rouge1'].mid.fmeasure],
    'rouge2': [scores['rouge2'].mid.precision, scores['rouge2'].mid.recall, scores['rouge2'].mid.fmeasure],
    'rougeL': [scores['rougeL'].mid.precision, scores['rougeL'].mid.recall, scores['rougeL'].mid.fmeasure],
}, index=[ 'P', 'R', 'F'])
df.style.format({'rouge1': "{:.4f}", 'rouge2': "{:.4f}", 'rougeL': "{:.4f}"})